In [1]:
import pandas as pd
from helpers import *

In [18]:
df = pd.read_csv('data/to_be_clustered.csv.gz', compression="gzip")
country = 'France'

In [15]:
df.columns

Index(['Unnamed: 0', 'created_at', 'text', 'source', 'translated', 'clean',
       'whcs', 'duplicated', 'language'],
      dtype='object')

In [8]:
categories = {}
categories['France'] = ['white/black - racism', 'islam', 'terrorism', 'coronavirus - vaccination', 
          'american elections', 'India - Pakistan', 'jew - antisemtism', 'farright', 'anti communism'
          'tweeter acccounts', 'fantasy sex play', 'Alain Soral', 'immigration', 'homophobia', 'other']

In [4]:
def prepare_xlsx_for_labelling(country, nb, text_column = 'translated'):
    df_c =pd.DataFrame(df[df.whcs == country][text_column])
    assert nb < len(df_c)
    df_c = df_c.sample(nb)
    for cat in categories[country]:
        df_c[cat] = 0
    df_c.to_excel('data/labelling/'+country+"_to_be_labeled.xlsx")

In [16]:
prepare_xlsx_for_labelling('France', 200, text_column = ['text', 'translated', 'source'])

## Get back the labeled tweets

In [44]:
df_lab = pd.read_excel('data/labelling/'+country+"_labeled.xlsx")
categories = ['white/black - racism',
       'islam', 'terrorism', 'coronavirus - vaccination', 'american elections',
       'India - Pakistan', 'jew - antisemtism', 'farright',
       'communism - against bankers , against state  prder ',
       'fantasy sex play', 'Alain Soral', 'immigration', 'homophobia',
       'tweeter acccounts', 'women', 'other', "Don't know"]
df_lab[categories] = df_lab[categories].astype(int)
df_lab.rename(columns={'Unnamed: 0' : 'index'}, inplace = True)
df_lab.set_index('index', inplace = True)

In [53]:
df_lab['sum'] = df_lab[categories].sum(axis = 1)
df_lab = df_lab[df_lab['sum'] > 0]
df_lab = df_lab[df_lab["Don't know"] != 1]
df_lab['labels'] = df_lab[categories].apply(lambda row: [col for col, b in zip(categories, row) if b],axis=1)
df_out = df_lab[['translated','labels']]

In [54]:
df_out.to_csv('data/labelling/'+country+"_to_be_labeled.xlsx"')

,translated,labels
index,,
12259,"Trump at CPAC: Trumpism means strong border, n...","[american elections, farright]"
19690,This post still has anti-Whites butthurt.#Blac...,[white/black - racism]
5455,Democrat Rep. Jackie Speier thinks a lot of wh...,[american elections]
6841,This is what happens to Whites when they becom...,[white/black - racism]
18285,WARNING: Attack in France is ComingMuslim Mayo...,"[islam, terrorism]"
...,...,...
9295,Female ‘Palestinian’ Muslim terrorist says: “P...,"[islam, jew - antisemtism]"
15165,Outside of environments that have been imposed...,[white/black - racism]
470,No Muslim ever respects a religious teacher of...,[islam]
